In [1]:
pip install fosforml

     |████████████████████████████████| 51kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 10.9MB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 83.9MB/s eta 0:00:01
     |████████████████████████████████| 307kB 102.0MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 93.7MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 128kB/s eta 0:00:012.4MB 1.4MB/s eta 0:01:37    |███████████████████████████████ | 194.2MB 1.3MB/s eta 0:00:056.2MB 1.3MB/s eta 0:00:04
     |████████████████████████████████| 98.7MB 156kB/s eta 0:00:011
     |████████████████████████████████| 133kB 106.9MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 89.3MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 67.3MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 655kB/s eta 0:00:011
     |████████████████████████████████| 51kB 17.8MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.8MB/s eta 0:00:01
     |███

In [2]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
df=sf_df.to_pandas()

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#How many % of missing values do we have for each feature?
missing_percentage = df.isnull().sum() / df.shape[0] * 100 
missing_percentage

In [ ]:
#no any transactions happened without distributer and no outliers/NULL in sales
df.loc[df.DISTRIBUTOR_CODE.isnull(), ["SALES_VOLUME", "SALES_UNITS"]].describe()

In [ ]:
#no any transactions happened without distributer and no outliers in sales
df.loc[df.OUTLET_CODE.isnull(), ["SALES_VOLUME", "SALES_UNITS"]].describe()

In [ ]:
df.OUTLET_CODE.nunique()

In [ ]:
df.DISTRIBUTOR_CODE.nunique()

In [ ]:
#no negative sales happened
[df.describe()<0]

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.corr(numeric_only=True)

In [ ]:

# Group by description and sum the sales
productby_sales = df.groupby('CATEGORY')['SALES_VOLUME'].sum().sort_values(ascending=False).iloc[0:30]

# Plotting
plt.figure(figsize=(20, 5))
sns.barplot(x=productby_sales.index, y=productby_sales.values, palette="Purples_r")
plt.ylabel("Total Sales")
plt.title("Which products has the most sales?")
plt.xticks(rotation=90)
plt.show()


In [ ]:

# Group by description and sum the sales
productby_qty = df.groupby('CATEGORY')['SALES_UNITS'].sum().sort_values(ascending=False).iloc[0:30]

# Plotting
plt.figure(figsize=(20, 5))
sns.barplot(x=productby_qty.index, y=productby_qty.values, palette="Greens_r")
plt.ylabel("Total Sales")
plt.title("Which products sold most?")
plt.xticks(rotation=90)
plt.show()


In [ ]:
df.COUNTY.nunique()

In [ ]:
# Find unique dates
df['TRANS_DATE'].nunique()

In [ ]:
state_counts = df.STATE.value_counts().sort_values(ascending=False).iloc[0:20]
plt.figure(figsize=(20,5))
sns.barplot(x=state_counts.index, y=state_counts.values, palette="Blues_r")
plt.ylabel("Counts")
plt.title("Which State made the most transactions?");
plt.xticks(rotation=90);
plt.yscale("log")

In [ ]:
#How many percentage of transactions happened in california
df.loc[df.STATE=="California"].shape[0] / df.shape[0] * 100

In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [ ]:
# Create features from date column
df['YEAR'] = df['TRANS_DATE'].dt.year
df['MONTH'] = df['TRANS_DATE'].dt.month
df['DAY'] = df['TRANS_DATE'].dt.day
df['DAY_OF_WEEK'] = df['TRANS_DATE'].dt.day_of_week + 1 # Monday is 1 and sunday is 7
df['QUARTER'] = df['TRANS_DATE'].dt.quarter
df['DAY_OF_YEAR'] = df['TRANS_DATE'].dt.dayofyear

In [ ]:
df.describe()

In [ ]:
df_unique = df.drop_duplicates()

In [ ]:
import matplotlib.pyplot as plt

# Plot SALES_UNITS over QUARTER
plt.figure(figsize=(10, 5))
df_unique.groupby('QUARTER')['SALES_UNITS'].sum().plot(kind='bar', color='orange')
plt.title('SALES_UNITS over QUARTER')
plt.xlabel('Quarter')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()

# Plot SALES_UNITS over MONTH
plt.figure(figsize=(10, 5))
df_unique.groupby('MONTH')['SALES_UNITS'].sum().plot(kind='bar', color='skyblue')
plt.title('SALES_UNITS over MONTH')
plt.xlabel('Month')
plt.ylabel('SALES_UNITS')
plt.grid(True)
plt.show()


In [ ]:
#most products are sold in quantities from 1 to 16
df.SALES_UNITS.describe()

In [ ]:
#uniform distribution
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(df.SALES_UNITS, ax=ax[0], kde=False, color="limegreen");
sns.distplot(np.log(df.SALES_UNITS), ax=ax[1], bins=20, kde=False, color="limegreen");
ax[0].set_title("Quantity distribution")
ax[0].set_yscale("log")
ax[1].set_title("Log-Quantity distribution")
ax[1].set_xlabel("Natural-Log Quantity");

In [ ]:
import datetime
      
startdate = datetime.datetime.strptime('2011-11-01', "%Y-%m-%d").date()
enddate = datetime.datetime.strptime('2011-11-01', "%Y-%m-%d").date()

In [ ]:
import pandas as pd

# Define the dates as strings
startdate_str = '2024-01-01'  # Replace with your actual start date
enddate_str = '2023-08-29'    # Replace with your actual end date

# Convert the strings to datetime objects
startdate = pd.to_datetime(startdate_str)
enddate = pd.to_datetime(enddate_str)

# Split the dataset into train and test dataset
df_test = df[df['TRANS_DATE'] > startdate] #1 month
df_train = df[df['TRANS_DATE'] <= enddate] # 11 month


In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
X_train = df_train.drop(labels=['MNTH_CODE','TRANS_DATE','START_DATE','SALES_PTR_VALUE','OC_CODE',
'DISTRIBUTOR_CODE','CITY','STATE','COUNTY','STREET','PRODUCT_CODE','SUBCATEGORY','BRAND','DAY','DAY_OF_WEEK','DAY_OF_YEAR'],axis=1)
y_train = df_train.SALES_UNITS

X_test = df_train.drop(labels=['MNTH_CODE','TRANS_DATE','START_DATE','SALES_PTR_VALUE','OC_CODE',
'DISTRIBUTOR_CODE','CITY','STATE','COUNTY','STREET','PRODUCT_CODE','SUBCATEGORY','BRAND','DAY','DAY_OF_WEEK','DAY_OF_YEAR'],axis=1)
y_test = df_train.SALES_UNITS



In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col_1 = df['OUTLET_CODE']
set_col_1 = list(set(col_1))
le.fit(col_1)
dict(zip(set_col_1, le.transform(set_col_1)))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col_2 = df['CATEGORY']
set_col_2 = list(set(col_2))
le.fit(col_2)
dict(zip(set_col_2, le.transform(set_col_2)))


In [ ]:
df.CATEGORY

In [ ]:
X_train.keys()

In [ ]:
X_test.keys()

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Exclude object type columns
le = preprocessing.LabelEncoder()
df['CATEGORY_ENCODED'] = le.fit_transform(df['CATEGORY'])
df['OUTLET_ENCODED'] = le.fit_transform(df['OUTLET_CODE'])

# Check the column names
print("Columns in DataFrame:", df.columns)

# Ensure the columns 'CATEGORY' and 'OUTLET_CODE' exist
if 'CATEGORY' in df.columns and 'OUTLET_CODE' in df.columns:
    # Drop the specified columns
    X = df.drop(['CATEGORY_ENCODED', 'OUTLET_ENCODED'], axis=1)

    # Separate the target variables
    y_category = df['CATEGORY']
    y_outlet_code = df['OUTLET_CODE']

    # If you need to combine them into a single DataFrame
    y = df['SALES_VOLUME']

    print(X.head())
    print(y.head())
else:
    print("Columns 'CATEGORY' and/or 'OUTLET_CODE' not found in DataFrame.")


In [ ]:
score = pd.DataFrame()
score.index.name = 'Scoring'
score['Training+Test Time(sec)'] = None
time_Frame = []

In [ ]:
df.dtypes

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
#data splitting

In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [ ]:
df['TRANS_DATE'] = df['TRANS_DATE'].apply(lambda x: x.timestamp())
df['START_DATE'] = df['START_DATE'].apply(lambda x: x.timestamp())

In [ ]:
df.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 42)
regressor.fit(X_train, y_train)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
%time regressor.fit(X_train, y_train)
%time regressor.fit(X_test, y_test)
y_pred_train = regressor.predict(X_train)
print('Train RSME :', np.sqrt(mean_squared_error(y_train, y_pred_train)))


y_pred_test = regressor.predict(X_test)
print('Test RSME :', np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col_1 = df['OUTLET_CODE']
set_col_1 = list(set(col_1))
le.fit(col_1)
dict(zip(set_col_1, le.transform(set_col_1)))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col_2 = df['CATEGORY']
set_col_2 = list(set(col_2))
le.fit(col_2)
dict(zip(set_col_2, le.transform(set_col_2)))

In [ ]:
df.CATEGORY

In [ ]:
X = df.drop('SALES_VOLUME',axis=1)
y = df['CATEGORY']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 42)
#X_train = X_train.astype('OBJECT')
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

In [ ]:
#less than 1 percent of the mean of all the values in the ‘sales_volume’ column
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#Accuracy of the prediction
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
import seaborn as sns
plt.figure(figsize=(5, 7))

ax = sns.distplot(y_test, hist=False, color="r", label="Actual Value")
sns.distplot(y_pred, hist=False, color="y", label="Fitted Values" , ax=ax)

plt.title('Actual vs Predicted Values Sales Volume')


plt.show()
plt.close()